In [7]:
import pandas as pd
data_train_cleaned = pd.read_pickle('data_train_after_cleanup')
data_test_cleaned = pd.read_pickle('data_test_after_cleanup')

In [8]:
from sklearn.model_selection import train_test_split
data_train_target = data_train_cleaned['Price']
data_train_final = data_train_cleaned.drop(['Name','Mileage','Engine','Power','New_Price','Price'],axis=1)

In [9]:
from sklearn.preprocessing import MinMaxScaler
for i in ['Kilometers_Driven','Mileage_cleaned','Engine_cleaned', 'Power_cleaned']:
    data_train_final[i] = MinMaxScaler(feature_range=(0,1)).fit_transform(data_train_final[i].values.astype(float).reshape(-1,1))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data_train_final, data_train_target, test_size=0.33)

In [11]:
import category_encoders as ce
ohe = ce.OneHotEncoder(handle_unknown='ignore',use_cat_names=True)
ohe.fit(X_train)
X_train_ohe=ohe.transform(X_train)

In [12]:
X_test_ohe=ohe.transform(X_test)

In [14]:
X_train_ohe.shape

(4032, 57)

In [18]:
for i in X_test_ohe.columns:
    if len(X_test_ohe[X_test_ohe[i].isnull()].index)>0:
        print(i,' Has Null values count : ',len(X_test_ohe[X_test_ohe[i].isnull()].index))

In [17]:
X_test_ohe = X_test_ohe.fillna(0)

<h2>Random Forest regressor model</h2>

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
hyperparameters = dict(n_estimators=[1, 2, 4, 8, 16, 32, 64, 100, 200],max_depth=np.linspace(1, 32, 32, endpoint=True),min_samples_split = np.linspace(0.1, 1.0, 10, endpoint=True),min_samples_leaf = np.linspace(0.1, 0.5, 5, endpoint=True))
clf=RandomizedSearchCV(RandomForestRegressor(), hyperparameters, random_state=1, cv=5, verbose=0,scoring='neg_mean_squared_error')
best_model = clf.fit(X_train_ohe, y_train)

In [ ]:
best_model

In [30]:
best_model.best_params_

{'n_estimators': 32,
 'min_samples_split': 0.1,
 'min_samples_leaf': 0.1,
 'max_depth': 3.0}

In [31]:
clf1= RandomForestRegressor(n_estimators=32,
 min_samples_split= 0.1,
 min_samples_leaf= 0.1,
 max_depth=3.0)
clf1.fit(X_train_ohe, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3.0,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.1, min_samples_split=0.1,
           min_weight_fraction_leaf=0.0, n_estimators=32, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [32]:
y_predict=clf1.predict(X_test_ohe)

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(y_test, y_predict))
print(rms)

7.059956385502494


<h2>Lasso regression model</h2>

In [34]:
from sklearn.linear_model import Lasso
C = [ 0.01,0.1,1, 10, 100, 1000]
param_grid = dict(alpha=C)
clf=RandomizedSearchCV(Lasso(), param_grid, random_state=1, cv=5, verbose=0,scoring='neg_mean_squared_error')
best_model = clf.fit(X_train_ohe, y_train)

C:\Users\LENOVO\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


In [35]:
best_model.best_params_

{'alpha': 0.01}

In [46]:
clf1= Lasso(alpha=0.001)
clf1.fit(X_train_ohe, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [47]:
from sklearn.metrics import mean_squared_error
from math import sqrt
y_predict=clf1.predict(X_test_ohe)
rms = sqrt(mean_squared_error(y_test, y_predict))
print(rms)

5.047663654330457
